This notebook creates the query images for Figure 1B, namely from layer t, batch 1.

# Save query images of bird-dog instruction trial

## Imports

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import time
import cv2

In [ ]:
import shutil
from torch.utils import data
from torchvision import transforms
from torchvision.datasets.folder import default_loader as default_image_loader

In [ ]:
import occlusion_utils as ut

# Natural Query Image

In [ ]:
# load image
# The two first authors decided to use the query image from channel t, batch 1.
query_path = "$DATAPATH/stimuli/stimuli_pure_conditions/channel/instruction_practice_catch/layer_t/kernel_size_t/channel_t/natural_images/batch_1/val"
data_loader = ut.get_data_loader(query_path)
image, _, _ = next(iter(data_loader))
image_np_transformed = image.numpy().transpose(0,2,3,1) # (1, 224, 224, 3)

In [ ]:
# Three options for the position of patch in maximal image.
# Option 1:
folder_suffix = _percent_side_length_dog_max_on_leg
start_positions_max = {}
# key: occlusion_size, value: tuple of x_start and y_start
start_positions_max[66] = (150, 110)
start_positions_max[90] = (134, 75)
start_positions_max[112] = (112, 52)

# Option 2:
folder_suffix = _percent_side_length_dog_max_on_body
start_positions_max = {}
# key: occlusion_size, value: tuple of x_start and y_start
start_positions_max[66] = (40, 50)
start_positions_max[90] = (50, 40)
start_positions_max[112] = (20, 20)

# Option 3:
# folder_suffix = _percent_side_length_dog_max_on_body_front
# start_positions_max = {}
# # key: occlusion_size, value: tuple of x_start and y_start
# start_positions_max[66] = (75, 130)
# start_positions_max[90] = (75, 115)
# start_positions_max[112] = (80, 100)

In [ ]:
# Position of patch in minimal image.
# no folder_suffix needed because it is defined in the max image
start_positions_min = {}
# key: occlusion_size, value: tuple of x_start and y_start
start_positions_min[66] = (5, 125)
start_positions_min[90] = (5, 112)
start_positions_min[112] = (3, 112)

In [ ]:
data_dir = "$DATAPATH/fig1_queries/"

# loop through occlusion sizes
for percentage_side_length_i, occlusion_size_i, heatmap_size_i in zip(
    ut.percentage_side_length_list, 
    ut.occlusion_sizes_list, 
    ut.heatmap_sizes_list):
    print(f"percentage_side_length_i {percentage_side_length_i}, occlusion_size_i {occlusion_size_i}, heatmap_size_i {heatmap_size_i}")

    query_dir = os.path.join(
        data_dir,
        f"{percentage_side_length_i}{folder_suffix}"
    )
    os.makedirs(query_dir, exist_ok=True)

    # loop through query images
    for query_type_i in ["default", "max_activation", "min_activation"]:
        print("query_type_i", query_type_i)

        # get images
        image_to_be_saved = copy.deepcopy(image_np_transformed.squeeze())

        # add patch to occlusion images
        # hand crafted by Judy
        if "activation" in query_type_i:
            if "min" in query_type_i:
                x_start, y_start = start_positions_min[occlusion_size_i]
                x_end, y_end = x_start + occlusion_size_i, y_start + occlusion_size_i
            elif "max" in query_type_i:
                x_start, y_start = start_positions_max[occlusion_size_i]
                x_end, y_end = x_start + occlusion_size_i, y_start + occlusion_size_i

            # add occlusion
            image_to_be_saved[x_start:x_end, y_start:y_end, :] = np.mean(np.mean(image_to_be_saved[x_start:x_end, y_start:y_end, :], axis=0), axis=0)

        image_path = os.path.join(query_dir, f"query_{query_type_i}.png")
#         cv2.imwrite(image_path, cv2.cvtColor(image_to_be_saved*255, cv2.COLOR_RGB2BGR))

        plt.imshow(image_to_be_saved)
        plt.show()